In [6]:
!pip install gensim spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 109.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# =========================================================
# 1. 라이브러리 설치 및 환경 설정
# =========================================================

import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaModel
import spacy
import os
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 경로 설정 (본인 환경에 맞게 수정)
base_path = "/content/drive/MyDrive/unstructured"
movie_final_path = f"{base_path}/movie_data_final_clean.csv"
save_path = f"{base_path}/lda_data"

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [9]:
# =========================================================
# 2. 데이터 로드 및 전처리 (공통 작업)
# =========================================================
print("📂 데이터 로드 및 전처리 시작...")

# 데이터 로드
df = pd.read_csv(movie_final_path)
df['overview'] = df['overview'].fillna('')
plots = df['overview'].tolist()

# Spacy 로드
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
stop_words = nlp.Defaults.stop_words.union({'movie', 'film', 'story', 'one', 'two', 'show', 'series'})

# 텍스트 전처리 함수
def process_texts(texts):
    final_tokens = []
    for doc in nlp.pipe(texts, batch_size=1000):
        tokens = [token.lemma_.lower() for token in doc
                  if token.pos_ in ['NOUN', 'ADJ', 'VERB']
                  and not token.is_stop
                  and token.lemma_.lower() not in stop_words
                  and len(token.lemma_) > 2]
        final_tokens.append(tokens)
    return final_tokens

# 전처리 수행
data_lemmatized = process_texts(plots)

# Bigram 적용
bigram = gensim.models.Phrases(data_lemmatized, min_count=5, threshold=10.0)
bigram_mod = gensim.models.phrases.Phraser(bigram)
data_final = [bigram_mod[doc] for doc in data_lemmatized]

# Dictionary & Corpus 생성
id2word = corpora.Dictionary(data_final)
# [중요] 단어 필터링 적용 (너무 흔하거나 희귀한 단어 제거)
id2word.filter_extremes(no_below=5, no_above=0.5)
corpus = [id2word.doc2bow(text) for text in data_final]

print(f"✅ 전처리 완료. 고유 단어 수: {len(id2word)}")

📂 데이터 로드 및 전처리 시작...
✅ 전처리 완료. 고유 단어 수: 5130


In [10]:
# =========================================================
# 3. LDA 학습 및 저장 (K=15, 20 반복)
# =========================================================
target_topics = [10]  # 생성할 토픽 수 리스트
THRESHOLD = 0.1           # 확률 임계값

for k in target_topics:
    print(f"\n🚀 [Topic K={k}] 학습 시작...")

    # 1. LDA 모델 학습
    lda_model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=k,
        random_state=42,
        update_every=1,
        chunksize=100,
        passes=10,
        alpha='auto',
        per_word_topics=True
    )

    # Coherence Score 확인
    coherence_model = CoherenceModel(model=lda_model, texts=data_final, dictionary=id2word, coherence='c_v')
    score = coherence_model.get_coherence()
    print(f"   -> Coherence Score: {score:.4f}")

    # 2. 엣지 데이터 생성 (Threshold Filtering)
    edge_data = []
    for i, doc_bow in enumerate(corpus):
        topic_dist = lda_model.get_document_topics(doc_bow, minimum_probability=0)

        # Threshold 이상인 토픽만 남김
        valid_topics = [(topic_id, prob) for topic_id, prob in topic_dist if prob >= THRESHOLD]

        for topic_id, prob in valid_topics:
            edge_data.append({
                'movie_id': i,       # 0 ~ 9011 (LightGCN Item ID와 매핑 필요)
                'topic_id': topic_id,
                'related': prob
            })

    # 3. CSV 저장
    edge_df = pd.DataFrame(edge_data)
    filename = f"topic_vectors_{k}.csv"
    save_file = os.path.join(save_path, filename)

    edge_df.to_csv(save_file, index=False)
    print(f"   💾 저장 완료: {filename} (연결 수: {len(edge_df):,}개)")

print("\n✨ 모든 작업 완료!")


🚀 [Topic K=10] 학습 시작...
   -> Coherence Score: 0.3285
   💾 저장 완료: topic_vectors_10.csv (연결 수: 26,272개)

✨ 모든 작업 완료!
